In [1]:
import sys
sys.path.append('../../../hypercat/hypercat/')
import hypercat

In [2]:
hdf5file = '/Volumes/Kohei_HDD_201803/hypercat/hypercat_20180417.hdf5'

In [3]:
cube = hypercat.ModelCube(hdf5file, hypercube='imgdata', subcube_selection='minimal')

Opening HDF5 file: /Volumes/Kohei_HDD_201803/hypercat/hypercat_20180417.hdf5 


OSError: Unable to open file (truncated file: eof = 75772788736, sblock->base_addr = 0, stored_eof = 745121888480)

'/Users/motigomeman/Study/ClumpyImage/hypercat/docs/notebooks'